# Important Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
# import contractions
import emoji
import string
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\LENOVO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
train_data = pd.read_csv('samsum-train.csv')
val_data = pd.read_csv('samsum-validation.csv')
test_data = pd.read_csv('samsum-test.csv')

In [5]:
train_data

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\r\...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\r\nSam:...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\r\nTheresa: <file_photo>...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\r\nJe...,Celia couldn't make it to the afternoon with t...


In [6]:
test_data

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....
...,...,...,...
814,13611902-1,Alex: Were you able to attend Friday night's b...,Benjamin didn't come to see a basketball game ...
815,13820989,Jamilla: remember that the audition starts at ...,The audition starts at 7.30 P.M. in Antena 3.
816,13717193,"Marta: <file_gif>\r\nMarta: Sorry girls, I cli...","Marta sent a file accidentally,"
817,13829115,Cora: Have you heard how much fuss British med...,There was a meet-and-greet with James Charles ...


In [7]:
val_data

,id,dialogue,summary
0,13817023,"A: Hi Tom, are you busy tomorrow’s afternoon?\...",A will go to the animal shelter tomorrow to ge...
1,13716628,Emma: I’ve just fallen in love with this adven...,Emma and Rob love the advent calendar. Lauren ...
2,13829420,Jackie: Madison is pregnant\r\nJackie: but she...,Madison is pregnant but she doesn't want to ta...
3,13819648,Marla: <file_photo>\r\nMarla: look what I foun...,Marla found a pair of boxers under her bed.
4,13728448,Robert: Hey give me the address of this music ...,Robert wants Fred to send him the address of t...
...,...,...,...
813,13829423,Carla: I've got it...\r\nDiego: what?\r\nCarla...,Carla's date for graduation is on June 4th. Di...
814,13727710,"Gita: Hello, this is Beti's Mum Gita, I wanted...",Bev is going on the school trip with her son. ...
815,13829261,"Julia: Greg just texted me\r\nRobert: ugh, del...",Greg cheated on Julia. He apologises to her. R...
816,13680226,"Marry: I broke my nail ;(\r\nTina: oh, no!\r\n...",Marry broke her nail and has a party tomorrow....


In [8]:
print(train_data.shape)
print(test_data.shape)
print(val_data.shape)

(14732, 3)
(819, 3)
(818, 3)


In [9]:
print(train_data.isnull().sum())
print(test_data.isnull().sum())
print(val_data.isnull().sum())

id          0
dialogue    1
summary     0
dtype: int64
id          0
dialogue    0
summary     0
dtype: int64
id          0
dialogue    0
summary     0
dtype: int64


In [10]:
train_data = train_data.dropna()

In [11]:
train_data = train_data.drop("id",axis=1)
test_data = test_data.drop("id",axis=1)
val_data = val_data.drop("id",axis=1)

In [12]:
stop_word = set(stopwords.words("english"))
negation_words = {
    "not", "no", "never", "none", "nobody", "nothing", "neither", "nowhere",
    "hasn't", "haven't", "hadn't", "doesn't", "don't", "didn't",
    "won't", "wouldn't", "can't", "couldn't", "isn't", "aren't", "wasn't", "weren't",
    "without", "nor"
}

filtered_stopwords = stop_word - negation_words

punctuation = set(string.punctuation)

def Preprocessing_text(text):

  text =text.lower()

  text = re.sub(r'<[^>]+>', '', text)

  text = re.sub(r"http\S+", "", text)

  text = re.sub(r"[^\w\s]", "", text)

  text = re.sub(r'\s+', ' ', text).strip()

  words = text.split()

  filtered = [word for word in words if word not in filtered_stopwords]

  return " ".join(filtered)

In [13]:
train_data['dialogue'] = train_data['dialogue'].apply(Preprocessing_text)
train_data['summary'] = train_data['summary'].apply(Preprocessing_text)

val_data['dialogue'] = val_data['dialogue'].apply(Preprocessing_text)
val_data['summary'] = val_data['summary'].apply(Preprocessing_text)

In [14]:
train_data

,dialogue,summary
0,amanda baked cookies want jerry sure amanda il...,amanda baked cookies bring jerry tomorrow
1,olivia voting election oliver liberals always ...,olivia olivier voting liberals election
2,tim hi whats kim bad mood tbh going lots stuff...,kim may try pomodoro technique recommended tim...
3,edward rachel think im ove bella rachel dont s...,edward thinks love bella rachel wants edward o...
4,sam hey overheard rick say something sam dont ...,sam confused overheard rick complaining roomma...
...,...,...
14727,romeo people may know list greta ah maybe chan...,romeo trying get greta add friend list refuses
14728,theresa theresa theresa hey louise u theresa w...,theresa work gets free food free yoga classes ...
14729,john every day bad news japan hunt whales eric...,japan going hunt whales island norway never st...
14730,jennifer dear celia jennifer afternoon collins...,celia couldnt make afternoon collins jennifer ...


In [15]:
val_data

,dialogue,summary
0,hi tom busy tomorrows afternoon b im pretty su...,go animal shelter tomorrow get puppy son alrea...
1,emma ive fallen love advent calendar awesome w...,emma rob love advent calendar lauren fits insi...
2,jackie madison pregnant jackie doesnt wanna ta...,madison pregnant doesnt want talk patricia ste...
3,marla marla look found bed kiki lol tamara som...,marla found pair boxers bed
4,robert hey give address music shop mentioned r...,robert wants fred send address music shop need...
...,...,...
813,carla ive got diego carla date graduation hope...,carlas date graduation june 4th diego try come
814,gita hello betis mum gita wanted ask going sch...,bev going school trip son gita going school tr...
815,julia greg texted robert ugh delete already ju...,greg cheated julia apologises robert tells jul...
816,marry broke nail tina oh no marry u know party...,marry broke nail party tomorrow tina call cosm...


In [16]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-small")
tokenizer = T5Tokenizer.from_pretrained("t5-small")


ImportError: 
T5ForConditionalGeneration requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFT5ForConditionalGeneration".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


In [ ]:
train_data["input_len"] = train_data["dialogue"].apply(lambda x: len(tokenizer.encode(x, truncation=False)))
val_data["input_len"] = val_data["dialogue"].apply(lambda x: len(tokenizer.encode(x, truncation=False)))

print("Train max:", train_data["input_len"].max())
print("Val max:", val_data["input_len"].max())


Token indices sequence length is longer than the specified maximum sequence length for this model (547 > 512). Running this sequence through the model will result in indexing errors


Train max: 563
Val max: 452


In [ ]:
max_input_len = train_data["input_len"].quantile(0.95)
max_target_len = train_data["summary"].apply(lambda x: len(tokenizer.encode(x, truncation=False))).quantile(0.95)
print(max_input_len)
print(max_target_len)

214.0
41.0


In [ ]:
from datasets import Dataset

train_data = train_data.reset_index(drop=True)
val_data = val_data.reset_index(drop=True)
test_data = test_data.reset_index(drop=True)


train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)
test_dataset = Dataset.from_pandas(test_data)


In [ ]:
def tokenization(text):
  return tokenizer(text["dialogue"] ,text["summary"],max_length=int(max_input_len),truncation=True,padding="max_length")


In [ ]:
def tokenization(example):
    model_inputs = tokenizer(
        example["dialogue"],
        max_length=int(max_input_len),
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            example["summary"],
            max_length=int(max_target_len),
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
tokenized_train = train_dataset.map(tokenization, batched=False)
tokenized_val = val_dataset.map(tokenization, batched=False)

Map:   0%|          | 0/14731 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir="./t5_summarizer",
    eval_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    metric_for_best_model="eval_loss",
    load_best_model_at_end=True,
    save_strategy="epoch",
    logging_steps=50,
    learning_rate=5e-5,
    weight_decay=0.01,
    save_total_limit=2,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

trainer.train()

Epoch,Training Loss,Validation Loss
